### ETL Gestión financiera - Mes a mes

En este notebook se realiza la ETL para los ingresos, gastos e indicadores de la compañía mes a mes de manera general. Cada mes se ejecuta la función final y genera una nueva fila con la información financiera general del mes para ser cargada con el script de Python a Azure Storage. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Se usa para que se muestren todas las columnas del dataframe
pd.options.display.max_columns=None

In [3]:
df=pd.read_csv('Auxiliar_General.csv')
df_enero=pd.read_csv('enero.csv')
df_febrero=pd.read_csv('febrero.csv')
df_marzo=pd.read_csv('marzo.csv')
df_abril=pd.read_csv('abril.csv')

con=pd.read_csv('Tabla_Conceptos_act.csv')

df_mayo=pd.read_csv('mayo.csv')
df_junio=pd.read_csv('junio.csv')
df_julio=pd.read_csv('julio.csv')

In [9]:
def financieras(df,con):
    
    #Crea la columna movimiento que corresponde a los débitos menos los créditos
    df['Movimiento']=df.apply(
    lambda row: row.Débitos - row.Créditos , axis=1)
    
    #Convierte la columna fecha a tipo datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    
    #Crea una nueva columna con la información de mes y año de la fecha registrada
    df['Mes']=df['Fecha'].dt.strftime('%m')
    
    #Convierte la columna 'Cuenta' a tipo object para poder realizar la busqueda de las cuentas 
    df['Cuenta']=df['Cuenta'].apply(str)
    
    #Crea un nuevo dataframe que resultada de la busqueda de las cuentas que empiezan con los dígitos: 4,5 o 7
    df_fil=df[df.Cuenta.str.startswith('4') | df.Cuenta.str.startswith('5') | df.Cuenta.str.startswith('7')].reset_index(drop=True)
    
    #Elimina unas cuentas especificas que no hacen parte de los resultados esperados
    df_fil.drop(df_fil[df_fil['Cuenta']=='7904950000'].index, inplace=True)
    df_fil.drop(df_fil[df_fil['Cuenta']=='5905010000'].index, inplace=True)
    df_fil=df_fil.reset_index(drop=True)
    
    #Crea una tabla dinamica desagregada por cuentas y año-mes
    pivotTable=pd.pivot_table(df_fil, values='Movimiento', index= 'Cuenta',
                   columns= 'Mes',
                   aggfunc= np.sum,
                   margins= False)
    
    #Convierte la columna 'Cuenta' de la tabla 'con' a tipo object
    con['Cuenta']=con['Cuenta'].apply(str)
    
    #Une la tabla pivotTable y la tabla con
    df_com=pd.merge(pivotTable, con, on='Cuenta', how='left')
    
    return df_com 

In [11]:
financieras(df,con)

,Cuenta,01,02,03,04,Nombre Cuenta,Conceptos PYG,Conceptos PFC,Rubro General,Rubro AOM,EBITDA,TIPO
0,4390900000,-1.245513e+09,-1.282834e+09,-1.529181e+09,-1.299023e+09,SERVICIOS BPO OPERACIÓN CENTRO,Ingresos Operacionales,Otros Ingresos con Empresas que No Consolidan,Ingresos operacionales,NO,NO,Ingresos_operacionales
1,4390900905,-4.056660e+08,-4.048993e+08,-6.652674e+08,-3.774175e+08,BPO,Ingresos Operacionales,Otros Ingresos con Empresas que No Consolidan,Ingresos operacionales,NO,NO,Ingresos_operacionales
2,4802010200,-1.296298e+07,-1.714181e+07,-1.901679e+07,-1.766267e+07,INT SOBRE DEP EN INST FINANCIE,Ingresos Financieros,Rendimientos Financieros,Ingresos financieros,NO,NO,Ingresos_no_operacionales
3,4806010100,NaN,-9.488950e+04,-2.272474e+05,-6.260598e+05,OT ING-DIF CAMBIO-EFECT-REALIZ,Ingresos Financieros,Rendimientos Financieros,Ingresos financieros,NO,NO,Ingresos_no_operacionales
4,4806020200,-4.259000e+04,3.418400e+04,-1.830320e+05,-4.312500e+04,OT ING-DIF CAMBIO-CXC-VALORADA,Ingresos No Operacionales,DC Ing-Cuentas x cobrar y pagar,Ingresos financieros,NO,NO,Ingresos_no_operacionales
...,...,...,...,...,...,...,...,...,...,...,...,...
128,7904070100,1.716604e+07,1.716604e+07,1.211985e+07,1.491367e+07,AMORTIZ LICENCIAS-CTOS,Depreciación y amortización - provisión,Total Deprec y Amortizaciones,Gastos operacionales,NO,SI,Gastos_Operacionales
129,7904070101,7.910750e+05,7.910750e+05,7.910750e+05,7.910750e+05,AMORTIZ SOFTWARE-CTOS,Depreciación y amortización - provisión,Total Deprec y Amortizaciones,Gastos operacionales,NO,SI,Gastos_Operacionales
130,7904070200,5.688988e+07,5.688988e+07,5.688988e+07,5.688988e+07,AMORTIZ CENTRO DE CONTROL,Depreciación y amortización - provisión,Total Deprec y Amortizaciones,Gastos operacionales,NO,SI,Gastos_Operacionales
131,7904081001,4.982051e+06,5.131335e+06,6.116726e+06,5.196092e+06,GRAVAMEN A LOS MOVIMIENTOS FIN,Contribuciones e impuestos,Impto- Tasas y Contribuciones,Gastos operacionales,Impuestos y contribuciones,SI,Gastos_Operacionales


### Calculos derivados de esta información:

In [12]:
def calculos(df,con,n):
    
    df_com=financieras(df,con)
    
    #Agrupa la tabla df_tipo solo por tipo , tomando como columnas el mes, que corresponde a df_columns[1]
    df_tipo=df_com.groupby('TIPO')[df_com.columns[1]].sum()
    
    #Renombra el indice
    df_tipo=df_tipo.rename_axis('')
    
    #El indice se nombra como 'Mes' y lo separa como una columna
    df_tipo=df_tipo.rename_axis('Mes').reset_index()
    
    #_________________________________________________________________________
    
    #Agrupa la tabla df_tipo solo por EBITDA , tomando como columnas el mes, que corresponde a df_columns[1]
    df_eb=df_com.groupby('EBITDA')[df_com.columns[1]].sum()
    
    #Renombra el indice
    df_eb=df_eb.rename_axis('')
    
    #El indice lo nombre como 'Mes' y lo separa como una columna
    df_eb=df_eb.rename_axis('Mes').reset_index()  
    #_________________________________________________________________________
    
    #Une por filas la tabla df_tipo y df_eb
    df_com=pd.concat([df_tipo,df_eb],ignore_index=True)
    
    #La columna 'Mes' pasa a ser el indice
    df_com.set_index('Mes',inplace=True)
    
    #Se transpone el dataframe
    df_com=df_com.transpose()
    
    #Elimina la columna 'NO'
    df_com=df_com.drop(['NO'], axis=1)
    
    #Cambia el nombre de la columna 'SI'
    df_com=df_com.rename(columns={'SI':'SUMA_SI_EBITDA'})
    
    # CALCULOS IMPORTANTES:
    
    df_com['UTILIDAD_NETA']=df_com['Gastos_Administrativos']+df_com['Gastos_Operacionales']+df_com['Gastos_no_operacionales']+df_com['Impuesto_Renta']+df_com['Ingresos_no_operacionales']+df_com['Ingresos_operacionales']
    
    df_com['UTILIDAD_BRUTA']=df_com['Gastos_Operacionales']+df_com['Ingresos_operacionales']
    
    df_com['UTILIDAD_OPERATIVA']=df_com['Gastos_Administrativos']+df_com['Gastos_Operacionales']+df_com['Ingresos_operacionales']
    
    df_com['UAI']=df_com['Gastos_Administrativos']+df_com['Gastos_Operacionales']+df_com['Gastos_no_operacionales']+df_com['Ingresos_no_operacionales']+df_com['Ingresos_operacionales']

    df_com['MARGEN_BRUTO']=df_com['UTILIDAD_BRUTA']/df_com['Ingresos_operacionales']
    df_com['MARGEN_OPERATIVO']=df_com['UTILIDAD_OPERATIVA']/df_com['Ingresos_operacionales']
    df_com['MARGEN_NETO']=df_com['UTILIDAD_NETA']/df_com['Ingresos_operacionales']
    
    df_com['EBITDA']=df_com['UTILIDAD_OPERATIVA']-df_com['SUMA_SI_EBITDA']
    df_com['MARGEN_EBITDA']=df_com['EBITDA']/df_com['Ingresos_operacionales']
    
    df_com['PartitionKey']='2022'
    df_com['RowKey']=n
    
    
    if n==1:
        df_com['MES']='01 Enero'
        
    elif n==2:
        df_com['MES']='02 Febrero'
        
    elif n==3:
        df_com['MES']='03 Marzo'
    
    elif n==4:
        df_com['MES']='04 Abril'
        
    elif n==5:
        df_com['MES']='05 Mayo'
        
    elif n==6:
        df_com['MES']='06 Junio'
        
    elif n==7:
        df_com['MES']='07 Julio'
        
    elif n==8:
        df_com['MES']='08 Agosto'
    
    elif n==9:
        df_com['MES']='09 Septiembre'
    
    elif n==10:
        df_com['MES']='10 Octubre'
    
    elif n==11:
        df_com['MES']='11 Noviembre'
        
    elif n==12:
        df_com['MES']='12 Diciembre'
        
    
        
    df_com=df_com[['PartitionKey', 'RowKey','MES','Gastos_Administrativos', 'Gastos_Operacionales',
       'Gastos_no_operacionales', 'Impuesto_Renta',
       'Ingresos_no_operacionales', 'Ingresos_operacionales',
       'UTILIDAD_NETA', 'UTILIDAD_BRUTA', 'UTILIDAD_OPERATIVA', 'UAI', 'EBITDA',
       'MARGEN_BRUTO', 'MARGEN_OPERATIVO', 'MARGEN_NETO',
       'MARGEN_EBITDA']]

           
    return df_com

In [13]:
calculos(df_enero,con,1)

Mes,PartitionKey,RowKey,MES,Gastos_Administrativos,Gastos_Operacionales,Gastos_no_operacionales,Impuesto_Renta,Ingresos_no_operacionales,Ingresos_operacionales,UTILIDAD_NETA,UTILIDAD_BRUTA,UTILIDAD_OPERATIVA,UAI,EBITDA,MARGEN_BRUTO,MARGEN_OPERATIVO,MARGEN_NETO,MARGEN_EBITDA
01,2022,1,01 Enero,3.195952e+08,976323949.4,9327004.69,147290102.0,-13005574.15,-1.651179e+09,-2.116482e+08,-674854948.6,-3.552598e+08,-3.589383e+08,-4.724733e+08,0.408711,0.215155,0.12818,0.286143


La siguiente linea toma la información de cada mes, para la cual realiza todos los calculos, y luego une todas estas tablas en una sola tabla: 

In [14]:
tabla2=pd.concat([calculos(df_enero,con,1),calculos(df_febrero,con,2),calculos(df_marzo,con,3),calculos(df_abril,con,4)],
                axis=0,ignore_index=True)


tabla2

Mes,PartitionKey,RowKey,MES,Gastos_Administrativos,Gastos_Operacionales,Gastos_no_operacionales,Impuesto_Renta,Ingresos_no_operacionales,Ingresos_operacionales,UTILIDAD_NETA,UTILIDAD_BRUTA,UTILIDAD_OPERATIVA,UAI,EBITDA,MARGEN_BRUTO,MARGEN_OPERATIVO,MARGEN_NETO,MARGEN_EBITDA
0,2022,1,01 Enero,3.195952e+08,9.763239e+08,9327004.69,1.472901e+08,-13005574.15,-1.651179e+09,-2.116482e+08,-6.748549e+08,-3.552598e+08,-3.589383e+08,-4.724733e+08,0.408711,0.215155,0.128180,0.286143
1,2022,2,02 Febrero,4.071303e+08,1.198039e+09,9734716.72,3.473190e+07,-17202513.68,-1.687733e+09,-5.529979e+07,-4.896942e+08,-8.256390e+07,-9.003169e+07,-1.962205e+08,0.290149,0.048920,0.032766,0.116263
2,2022,3,03 Marzo,5.708849e+08,9.974558e+08,9204173.93,2.575872e+08,-19427065.34,-2.194449e+09,-3.787438e+08,-1.196993e+09,-6.261081e+08,-6.363310e+08,-7.477065e+08,0.545464,0.285315,0.172592,0.340726
3,2022,4,04 Abril,6.463163e+08,8.992648e+08,18597909.81,5.408471e+07,-18331855.21,-1.676441e+09,-7.650891e+07,-7.771759e+08,-1.308597e+08,-1.305936e+08,-2.494170e+08,0.463587,0.078058,0.045638,0.148778


In [15]:
vwNewIndicadoresGeneral=tabla2
vwNewIndicadoresGeneral.to_csv('vwNewIndicadoresGeneral.csv', index=False)

La siguiente línea ejecuta las funciones para la carga de información de mayo, junio y julio:

In [19]:
tabla3=pd.concat([calculos(df_mayo,con,5),calculos(df_junio,con,6),calculos(df_julio,con,7)],axis=0,ignore_index=True)
tabla3

Mes,PartitionKey,RowKey,MES,Gastos_Administrativos,Gastos_Operacionales,Gastos_no_operacionales,Impuesto_Renta,Ingresos_no_operacionales,Ingresos_operacionales,UTILIDAD_NETA,UTILIDAD_BRUTA,UTILIDAD_OPERATIVA,UAI,EBITDA,MARGEN_BRUTO,MARGEN_OPERATIVO,MARGEN_NETO,MARGEN_EBITDA
0,2022,5,05 Mayo,4.783152e+08,9.706309e+08,9894111.36,1.683606e+08,-42852169.46,-1.820043e+09,-2.356946e+08,-8.494124e+08,-3.710972e+08,-4.040553e+08,-4.921517e+08,0.466699,0.203895,0.129499,0.270407
1,2022,6,06 Junio,6.690979e+08,8.676537e+08,28999646.18,-9.622601e+07,-39754968.68,-1.308140e+09,1.216301e+08,-4.404864e+08,2.286115e+08,2.178562e+08,1.044684e+08,0.336727,-0.174761,-0.092979,-0.079860
2,2022,7,07 Julio,5.630262e+08,1.125923e+09,22781144.86,2.502536e+07,-40313186.54,-1.720966e+09,-2.452345e+07,-5.950429e+08,-3.201677e+07,-4.954881e+07,-1.656834e+08,0.345761,0.018604,0.014250,0.096274


In [20]:
tabla3.to_csv('vwNewIndicadoresGeneralMay_July.csv', index=False)